In [ ]:
import ccxt
import time
from dotenv import load_dotenv
import os
import csv
from datetime import datetime
import pandas as pd
import logging

# Configure logging
logging.basicConfig(filename='obi_arb.log', level=logging.ERROR)
load_dotenv()

class KuCoinFuturesClient:
    def __init__(self, api_key, api_secret, api_password):
        self.exchange = ccxt.kucoinfutures({
            'apiKey': os.getenv('API_KEY'),
            'secret': os.getenv('SECRET_KEY'),
            'password': os.getenv('PASSPHRASE'),
            'enableRateLimit': True  # Adjust as needed
        })
        self.symbol = 'ARB/USDT:USDT'
        self.LEVERAGE = 5
        self.AMOUNT = 1

class TradingSignalGenerator:
    def __init__(self, threshold_negative=-0.08, threshold_positive=0.08):
        self.threshold_negative = threshold_negative
        self.threshold_positive = threshold_positive

    def generate_signal(self, smoothed_imbalance, ema_cross_signal):
        if smoothed_imbalance < self.threshold_negative and ema_cross_signal == "Selling Cross":
            return "Strong Sell"
        elif smoothed_imbalance > self.threshold_positive and ema_cross_signal == "Buying Cross":
            return "Strong Buy"
        else:
            return "No Signal"

class MarketOrderManager:
    def __init__(self, kucoin_client):
        self.exchange = kucoin_client.exchange
        self.symbol = kucoin_client.symbol
        self.leverage = kucoin_client.LEVERAGE
        self.amount = kucoin_client.AMOUNT

    def has_open_orders(self):
        try:
            orders = self.exchange.fetch_open_orders(symbol=self.symbol)
            if orders:
                print(f"There are already open orders for {self.symbol}. Aborting new order creation.")
            else:
                print(f"Symbol {self.symbol} ready to be traded.")
            return bool(orders)
        except ccxt.NetworkError as e:
            logging.error(f'Network error while checking open orders: {e}')
            return False
        except ccxt.ExchangeError as e:
            logging.error(f'Exchange error while checking open orders: {e}')
            return False
        except Exception as e:
            logging.error(f'An error occurred while checking open orders: {e}')
            return False

    def close_open_orders(self):
        try:
            orders = self.exchange.fetch_open_orders(symbol=self.symbol)
            for order in orders:
                closed_order = self.exchange.cancel_order(order['id'], symbol=self.symbol)
                print(f"Closed Order {closed_order['id']}")
        except ccxt.NetworkError as e:
            logging.error(f'Network error while closing open orders: {e}')
        except ccxt.ExchangeError as e:
            logging.error(f'Exchange error while closing open orders: {e}')
        except Exception as e:
            logging.error(f'An error occurred while closing open orders: {e}')

    def create_market_order(self, side, amount):
        if self.has_open_orders():
            print(f"There are already open orders for {self.symbol}. Aborting new order creation.")
            return None

        try:
            print(f"DEBUG: create_market_order - side: {side}, symbol: {self.symbol}, amount: {self.amount}, leverage: {self.leverage}")

            market_order = self.exchange.create_order(
                symbol=self.symbol,
                type='market',
                side=side,
                amount=amount,
                params={'timeInForce': 'GTC', 'leverage': self.leverage}
            )

            print(f"Main Order Created: {market_order}")

            current_ema_cross_signal = None

            while True:
                ema_cross_signal = self.detect_ema_cross(short_ema, long_ema)  # Define short_ema and long_ema
                if ema_cross_signal != current_ema_cross_signal:
                    self.close_open_orders()
                    if ema_cross_signal == "Buying Cross" and not self.has_open_orders():
                        self.create_market_order('buy', current_price)
                    elif ema_cross_signal == "Selling Cross" and not self.has_open_orders():
                        self.create_market_order('sell', current_price)
                    current_ema_cross_signal = ema_cross_signal
                time.sleep(20)

            return market_order

        except ccxt.NetworkError as e:
            logging.error(f'Network error: {e}')
            return None
        except ccxt.ExchangeError as e:
            logging.error(f'Exchange error: {e}')
            return None
        except Exception as e:
            logging.error(f'An error occurred: {e}')
            return None

class CurrentPricePrinter:
    def __init__(self, kucoin_client, order_manager, signal_generator, alpha=0.1, short_ema_period=9, long_ema_period=26):
        self.exchange = kucoin_client.exchange
        self.symbol = kucoin_client.symbol
        self.alpha = alpha
        self.short_ema_period = short_ema_period
        self.long_ema_period = long_ema_period
        self.csv_file_path = 'obi_arb.csv'
        self.signal_generator = signal_generator
        self.order_manager = order_manager

    def calculate_smoothed_imbalance(self, data):
        try:
            df = pd.DataFrame(data, columns=['imbalance'])
            order_book = self.exchange.fetch_order_book(self.symbol)
            df_order_book = pd.DataFrame(order_book['bids'], columns=['bid_price', 'bid_volume'])
            df_order_book['ask_price'] = [ask[0] for ask in order_book['asks']]
            df_order_book['ask_volume'] = [ask[1] for ask in order_book['asks']]
            total_bid_value = (df_order_book['bid_price'] * df_order_book['bid_volume']).sum()
            total_ask_value = (df_order_book['ask_price'] * df_order_book['ask_volume']).sum()
            total_order_book_value = total_bid_value + total_ask_value
            imbalance_usdt_value = total_order_book_value * df['imbalance'].iloc[-1]
            df['smoothed_imbalance'] = df['imbalance'].ewm(alpha=self.alpha, adjust=False).mean()
            print(f'Imbalance USDT Value: {imbalance_usdt_value:.2f}')
            return df['smoothed_imbalance'].tolist(), imbalance_usdt_value
        except Exception as e:
            logging.error(f"Error calculating smoothed imbalance: {e}")
            return None, None

    def detect_ema_cross(self, short_ema, long_ema):
        if short_ema > long_ema:
            return "Buying Cross"
        elif short_ema < long_ema:
            return "Selling Cross"
        else:
            return "No Cross"

    def print_current_price(self):
        if not os.path.exists(self.csv_file_path):
            with open(self.csv_file_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(['Current Time', 'Current Price', 'Imbalance USDT Value', 'Smoothed Imbalance', 'EMA Cross Signal', 'Trading Signal'])

        while True:
            try:
                ohlcv = self.exchange.fetch_ohlcv(self.symbol, timeframe='1m', limit=1)
                df_ohlcv = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df_ohlcv['timestamp'] = pd.to_datetime(df_ohlcv['timestamp'], unit='ms')
                current_price = df_ohlcv['close'].iloc[-1]

                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'\nCurrent Time: {current_time}')
                print(f'Current Price for {self.symbol}: {current_price} USDT')

                order_book = self.exchange.fetch_order_book(self.symbol)
                df_order_book = pd.DataFrame(order_book['bids'], columns=['bid_price', 'bid_volume'])
                df_order_book['ask_price'] = [ask[0] for ask in order_book['asks']]
                df_order_book['ask_volume'] = [ask[1] for ask in order_book['asks']]

                asks_total = df_order_book['ask_volume'].sum()
                bids_total = df_order_book['bid_volume'].sum()
                order_book_imbalance = (asks_total - bids_total) / (asks_total + bids_total)
                smoothed_imbalance, imbalance_usdt_value = self.calculate_smoothed_imbalance([order_book_imbalance])

                if smoothed_imbalance:
                    print(f'Smoothed Imbalance for {self.symbol}: {smoothed_imbalance[0]:.4f}')

                # Calculate short EMA (9 periods)
                short_ema_period = 9
                short_ema = df_ohlcv['close'].ewm(span=short_ema_period, adjust=False).mean().iloc[-1]

                # Calculate long EMA (26 periods)
                long_ema_period = 26
                long_ema = df_ohlcv['close'].ewm(span=long_ema_period, adjust=False).mean().iloc[-1]

                cross_signal = self.detect_ema_cross(short_ema, long_ema)
                print(f'EMA Cross Signal: {cross_signal}')

                trading_signal = self.signal_generator.generate_signal(smoothed_imbalance[0], cross_signal)
                print(f'Trading Signal: {trading_signal}')

                self.order_manager.close_open_orders()

                if trading_signal == "Strong Buy" and not self.order_manager.has_open_orders():
                    self.order_manager.create_market_order('buy', current_price)
                elif trading_signal == "Strong Sell" and not self.order_manager.has_open_orders():
                    self.order_manager.create_market_order('sell', current_price)

                with open(self.csv_file_path, 'a', newline='') as csvfile:
                    csv_writer = csv.writer(csvfile)
                    csv_writer.writerow([current_time, current_price, imbalance_usdt_value, smoothed_imbalance[0], cross_signal, trading_signal])

            except ccxt.NetworkError as e:
                logging.error(f'Network error: {e}')
            except ccxt.ExchangeError as e:
                logging.error(f'Exchange error: {e}')
            except ValueError as e:
                logging.error(f'Value error: {e}')
            except KeyError as e:
                logging.error(f'Key error: {e}')
            except Exception as e:
                logging.error(f'An error occurred: {e}')

            time.sleep(10)

def main():
    # Load API credentials from environment variables
    api_key = os.getenv('KUCOIN_API_KEY')
    api_secret = os.getenv('KUCOIN_API_SECRET')
    api_password = os.getenv('PASSPHRASE')

    # Instantiate the KuCoinFuturesClient
    kucoin_client = KuCoinFuturesClient(api_key, api_secret, api_password)

    # Instantiate the TradingSignalGenerator
    signal_generator = TradingSignalGenerator()

    # Instantiate the MarketOrderManager
    order_manager = MarketOrderManager(kucoin_client)

    # Instantiate the CurrentPricePrinter
    price_printer = CurrentPricePrinter(kucoin_client, order_manager, signal_generator)

    # Run the function
    price_printer.print_current_price()

if __name__ == "__main__":
    main()



Current Time: 2023-10-30 10:27:32
Current Price for ARB/USDT:USDT: 0.9429 USDT
Imbalance USDT Value: 87444.71
Smoothed Imbalance for ARB/USDT:USDT: 0.2563
EMA Cross Signal: No Cross
Trading Signal: No Signal

Current Time: 2023-10-30 10:27:42
Current Price for ARB/USDT:USDT: 0.9429 USDT
Imbalance USDT Value: -88224.82
Smoothed Imbalance for ARB/USDT:USDT: -0.2921
EMA Cross Signal: No Cross
Trading Signal: No Signal

Current Time: 2023-10-30 10:27:53
Current Price for ARB/USDT:USDT: 0.9429 USDT
Imbalance USDT Value: -72822.22
Smoothed Imbalance for ARB/USDT:USDT: -0.2914
EMA Cross Signal: No Cross
Trading Signal: No Signal

Current Time: 2023-10-30 10:28:44
Current Price for ARB/USDT:USDT: 0.9413 USDT
Imbalance USDT Value: -49791.72
Smoothed Imbalance for ARB/USDT:USDT: -0.2135
EMA Cross Signal: No Cross
Trading Signal: No Signal
